## Read w3resource exercise group pages

In [2]:
import os
from bs4 import BeautifulSoup
import requests
import pymongo
import re
import sys
import json

In [3]:
# Get groups from database

# Open w3resource database
client = pymongo.MongoClient('mongodb://localhost:27017/')
db = client['w3resource-python']

# Get group of exercises collection
colGroups = db["groups-exercises"]
colExercises = db["exercises"]

# groups = []
# for group in colGroups.find().sort("sequence"):
#    groups.append(group)

group_id = 'Python Basic (Part -II)'
group = colGroups.find_one({"_id": group_id})

In [4]:
# Get url and url path
url = group['url']

path_regex = re.compile('^.+/')
url_path = path_regex.search(url).group()

In [5]:
import json
# print(json.dumps(groups, indent=4))

In [6]:
# Exercises array
exercises = []

# Store  exercise in exercises
def store_exercise(sequence, statement, instructions, solution_url):

   # Get solution
   response = requests.get(solution_url, headers=headers)
   bs = BeautifulSoup(response.text, 'lxml')

   # match = bs.find('code', {'class': {'language-bash', 'language-python', 'language-Python', 'language-html', 'language-sql'}})
   # match = bs.find('pre', {'class': {'line-numbers'}})
   match = bs.find('code', {'class': {'language-python', 'language-Python', 'language-html', 'language-sql'}})
   solution = ''
   if match is not None:
      solution = match.text

   exercises.append({
      'group': group_id,
      'sequence': sequence,
      'statement': statement,
      'instructions': instructions,
      'solution_url': solution_url,
      'solution': solution,
      'important': False,
      'solved': False
   })

In [7]:
headers = {
   'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0',
   'X-Requested-With': 'XMLHttpRequest'
}

response = requests.get('https://www.w3resource.com/python-exercises/python-basic-exercise-46.php', headers=headers)
bs = BeautifulSoup(response.text, 'lxml')
match = bs.find('pre', {'class': {'line-numbers'}})
print(match is None)
match.text

False


'import os\r\nprint("Current File Name : ",os.path.realpath(__file__))\r\n\r\n'

In [38]:
# Get exercises per group

headers = {
   'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0',
   'X-Requested-With': 'XMLHttpRequest'
}

# Read w3resource group page
response = requests.get(url, headers=headers)
bs = BeautifulSoup(response.text, 'lxml')

exercises = []
sequence = 0
header = ''

for element in bs.find('h2', {'class':'heading'}).next_siblings:

   if element.name is not None:

      # Exclude p.heading element
      if 'class' in element.attrs and 'heading' in element.attrs['class']:
         continue

      # Wrap <a> tag element with <p> tag
      if element.name == 'a':
         bs2 = BeautifulSoup(str(element), 'lxml')
         element = bs2.a.wrap(bs2.new_tag('p'))

      # get solution page url
      solution_url = None
      match = element.find('a', text='Click me to see the sample solution')
      if match != None:
         solution_url = url_path + match.attrs['href']
         # Get rid of solution url link
         element.find('a', text='Click me to see the sample solution').decompose()

      # Test for statement line 
      match2 = re.search('^<p><strong>([0-9]+).*<\/strong>', str(element))
      if match2: # statement line

         # Get exercise sequence
         sequence = int(match2.group(1))

         # Get rid of first strong tag
         element.find('strong').decompose()

         # Get rid of editor link
         if element.find(href='#EDITOR') is not None:
            element.find(href='#EDITOR').decompose()

         # Get rid of wrapping p tag
         content = str(element)
         content = re.sub('^<[pP]>', '', content)
         content = re.sub('</[pP]>$', '', content)

         # Get exercise statement and rest of the content
         content = re.split('<br[/]*>', content, 1)
         statement = content[0]
         instructions = ''
         if len(content) == 2:
            instructions = content[1]

      else:

         if element.name == 'pre':
            instructions += '<pre>' + str(element.find(text=True)) + '</pre>'
         else:
            instructions += str(element)

      # Save exercise data
      if solution_url is not None:
         # strip <p></p> tags if it exists in instructions
         instructions = re.sub('<[pP]></[pP]>', '', instructions)
         store_exercise(sequence, statement, instructions, solution_url)

   else:
         # Get header
         if sequence == 0:
            header += str(element)
         else:
            instructions += str(element)


   # print(element)

   if element.name != None and 'id' in element.attrs:
      # print(element.attrs['id'])
      break
   

print(json.dumps(exercises, indent=4))



[
    {
        "group": "Python Basic (Part -II)",
        "sequence": 1,
        "statement": " Write a Python function that takes a sequence of numbers and determines whether all the numbers are different from each other. ",
        "instructions": "\n",
        "solution_url": "https://www.w3resource.com/python-exercises/basic/python-basic-1-exercise-1.php",
        "solution": "def test_distinct(data):\r\n  if len(data) == len(set(data)):\r\n    return True\r\n  else:\r\n    return False;\r\nprint(test_distinct([1,5,7,9]))\r\nprint(test_distinct([2,4,5,5,7,9]))\r\n",
        "important": false,
        "solved": false
    },
    {
        "group": "Python Basic (Part -II)",
        "sequence": 2,
        "statement": " Write a Python program to create all possible strings by using 'a', 'e', 'i', 'o', 'u'. Use the characters exactly once. ",
        "instructions": "\n",
        "solution_url": "https://www.w3resource.com/python-exercises/basic/python-basic-1-exercise-2.php",
     

#### Verify if all exercises were loaded correctly

In [39]:
# print(json.dumps(exercises, indent=4))

group_test = []
sequence_test = []
statement_test = []
solution_url_test = []
solution_test = []
index = 1

for item in exercises:

   if item['group'] != group_id:
      group_test.append(item['sequence'])

   if item['sequence'] > index:
      for i in range(index, item['sequence']):
         index += 1
         sequence_test.append(i)

   if item['statement'] == '':
      statement_test.append(item['sequence'])

   if item['solution_url'] == '':
      solution_url_test.append(item['sequence'])
      
   if item['solution'] == '':
      solution_test.append(item['sequence'])

   index += 1

# Print results
if len(group_test) == 0:
   print('group: ok')
else:
   print('missing groups: ' + str(group_test) )

if len(sequence_test) == 0:
   print('sequences: ok')
else:
   print('missing sequences: ' + str(sequence_test) )

if len(statement_test) == 0:
   print('statement: ok')
else:
   print('missing statements: ' + str(statement_test) )

if len(solution_url_test) == 0:
   print('solution_url: ok')
else:
   print('missing solution_urls: ' + str(solution_url_test) )

if len(solution_test) == 0:
   print('solutions: ok')
else:
   print('missing solutions: ' + str(solution_test) )



group: ok
sequences: ok
statement: ok
solution_url: ok
solutions: ok


#### Save exercises in the database

In [40]:
# Delete existing records
query = { "group": group_id }
colExercises.delete_many(query)

# insert exercises
x = colExercises.insert_many(exercises)

#print list of the _id values of the inserted documents:
print(x.inserted_ids)

[ObjectId('62e1e7211c17aadc41bca1f8'), ObjectId('62e1e7211c17aadc41bca1f9'), ObjectId('62e1e7211c17aadc41bca1fa'), ObjectId('62e1e7211c17aadc41bca1fb'), ObjectId('62e1e7211c17aadc41bca1fc'), ObjectId('62e1e7211c17aadc41bca1fd'), ObjectId('62e1e7211c17aadc41bca1fe'), ObjectId('62e1e7211c17aadc41bca1ff'), ObjectId('62e1e7211c17aadc41bca200'), ObjectId('62e1e7211c17aadc41bca201'), ObjectId('62e1e7211c17aadc41bca202'), ObjectId('62e1e7211c17aadc41bca203'), ObjectId('62e1e7211c17aadc41bca204'), ObjectId('62e1e7211c17aadc41bca205'), ObjectId('62e1e7211c17aadc41bca206'), ObjectId('62e1e7211c17aadc41bca207'), ObjectId('62e1e7211c17aadc41bca208'), ObjectId('62e1e7211c17aadc41bca209'), ObjectId('62e1e7211c17aadc41bca20a'), ObjectId('62e1e7211c17aadc41bca20b'), ObjectId('62e1e7211c17aadc41bca20c'), ObjectId('62e1e7211c17aadc41bca20d'), ObjectId('62e1e7211c17aadc41bca20e'), ObjectId('62e1e7211c17aadc41bca20f'), ObjectId('62e1e7211c17aadc41bca210'), ObjectId('62e1e7211c17aadc41bca211'), ObjectId('6